<a href="https://colab.research.google.com/github/lumiaresilva/lumiare/blob/main/isma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install library yang dibutuhkan
!pip install scikit-image scikit-learn

# Import library yang diperlukan
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops # The function name is 'graycomatrix', not 'greycomatrix'
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Fungsi untuk ekstraksi fitur menggunakan GLCM
def extract_glcm_features(image):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # GLCM Parameters
    distances = [1]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    # Compute GLCM
    glcm = graycomatrix(gray_image, distances, angles, 256, symmetric=True, normed=True) # The function name is 'graycomatrix', not 'greycomatrix'
    # Extract GLCM properties
    contrast = graycoprops(glcm, 'contrast').flatten() # The function name is 'graycoprops', not 'greycoprops'
    dissimilarity = graycoprops(glcm, 'dissimilarity').flatten() # The function name is 'graycoprops', not 'greycoprops'
    homogeneity = graycoprops(glcm, 'homogeneity').flatten() # The function name is 'graycoprops', not 'greycoprops'
    energy = graycoprops(glcm, 'energy').flatten() # The function name is 'graycoprops', not 'greycoprops'
    correlation = graycoprops(glcm, 'correlation').flatten() # The function name is 'graycoprops', not 'greycoprops'
    # Return all properties as a single feature vector
    return np.hstack([contrast, dissimilarity, homogeneity, energy, correlation])

# Path ke folder dataset
data_path = "oysters/"

# List semua folder kategori (baik dan buruk)
categories = ["good", "bad"]

# Inisialisasi list untuk fitur dan label
features = []
labels = []

# Loop melalui setiap kategori untuk memproses gambar
for category in categories:
    folder_path = os.path.join(data_path, category)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        # Membaca gambar
        image = cv2.imread(file_path)
        if image is not None:
            # Ekstrak fitur GLCM
            feature = extract_glcm_features(image)
            features.append(feature)
            labels.append(category)

# Konversi fitur dan label ke array
X = np.array(features)
y = np.array(labels)

# Encode label (baik dan buruk menjadi 0 dan 1)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Inisialisasi model KNN
knn = KNeighborsClassifier(n_neighbors=3)

# Training model
knn.fit(X_train, y_train)

# Prediksi pada data testing
y_pred = knn.predict(X_test)

# Evaluasi model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nAccuracy Score:", accuracy_score(y_test, y_pred))

Confusion Matrix:
[[11  1]
 [ 0 20]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        12
           1       0.95      1.00      0.98        20

    accuracy                           0.97        32
   macro avg       0.98      0.96      0.97        32
weighted avg       0.97      0.97      0.97        32


Accuracy Score: 0.96875


In [ ]:
# Fungsi untuk memuat dan menguji citra tunggal menggunakan model KNN
def test_single_image(image_path, model, scaler, label_encoder):
    # Membaca gambar
    image = cv2.imread(image_path)
    if image is not None:
        # Ekstrak fitur GLCM dari citra
        features = extract_glcm_features(image)
        # Normalisasi fitur
        features = scaler.transform([features])
        # Prediksi menggunakan model KNN
        prediction = model.predict(features)
        # Decode hasil prediksi ke label
        predicted_label = label_encoder.inverse_transform(prediction)
        return predicted_label[0]
    else:
        print("Gambar tidak ditemukan.")
        return None

# Path ke citra uji
test_image_path = "oysters/test/test.png"  # Ganti path ini dengan path citra uji Anda

# Uji model pada citra tunggal
predicted_quality = test_single_image(test_image_path, knn, scaler, label_encoder)

# Tampilkan hasil prediksi
if predicted_quality is not None:
    print(f"Hasil prediksi kualitas tiram pada citra uji: {predicted_quality}")


Hasil prediksi kualitas tiram pada citra uji: good


In [ ]:
# Simpan model dan skaler setelah pelatihan
import joblib

# Simpan model
joblib.dump(knn, 'models/knn_model.pkl')
# Simpan skaler
joblib.dump(scaler, 'models/scaler.pkl')


['models/scaler.pkl']